In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import datetime
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")
pyproj.datadir.set_data_dir("D:/ProgramData/anaconda3/envs/crime-prediction/Library/share/proj")

In [2]:
pre_processed_file_path = 'data/preprocessed/preprocessed-2.csv'
poverty_data_file_path = 'data/preprocessed/poverty_data.geojson'

In [3]:
def parse_preprocessed_data():
    df = pd.read_csv(
                         pre_processed_file_path,
                         parse_dates=['Offense Date'],                       
                         dtype={
                             'Precinct': 'category',
                             'Offense Category': 'category',
                             'Latitude': np.float32,
                             'Longitude': np.float32
                         }
                        )
    gdf = gpd.GeoDataFrame(df, geometry= df["geometry"].apply(wkt.loads), crs='EPSG:4326')
    return gdf

In [12]:
crime_gdf = parse_preprocessed_data()

In [8]:
poverty_gdf = gpd.read_file(poverty_data_file_path)

In [13]:
print(poverty_gdf.columns)
print(crime_gdf.columns)

Index(['Tract Name', 'Tract Population', 'Tract Population Below Poverty',
       'geometry'],
      dtype='object')
Index(['Offense Date', 'Latitude', 'Longitude', 'Beat', 'Precinct', 'Sector',
       'Neighborhood', 'Reporting Area', 'Offense Category', 'geometry',
       'centroid', 'Temperature', 'Wind Speed', 'Precipitation', 'Visibility'],
      dtype='object')


In [14]:
crime_gdf = gpd.sjoin(crime_gdf, poverty_gdf, how="left", predicate="within") \
                .drop(columns=['index_right'])
print(crime_gdf.columns)

Index(['Offense Date', 'Latitude', 'Longitude', 'Beat', 'Precinct', 'Sector',
       'Neighborhood', 'Reporting Area', 'Offense Category', 'geometry',
       'centroid', 'Temperature', 'Wind Speed', 'Precipitation', 'Visibility',
       'Tract Name', 'Tract Population', 'Tract Population Below Poverty'],
      dtype='object')


In [16]:
crime_gdf.to_csv('data/preprocessed/preprocessed-3.csv', index=False)